After going thorugh metadata, I found out the listings' prices are stored relevant to each country local currency, in order to use the price column in my dashboard, and becides transformation I made using Mage, I decided to use spark and pyspark this time to convert all prices to usd.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import avg
from pyspark.sql.types import FloatType
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

credentials_location = '/home/sal/git/data-engineering-capstone/spark/keys/my-creds.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('airbnb') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

In [2]:
spark = SparkSession.builder \
  .appName('Airbnb') \
  .config('spark.jars.packages', 'com.google.cloud.spark:spark-3.3-bigquery:0.37.0') \
  .getOrCreate()

:: loading settings :: url = jar:file:/home/sal/spark/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sal/.ivy2/cache
The jars for the packages stored in: /home/sal/.ivy2/jars
com.google.cloud.spark#spark-3.3-bigquery added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-18fc1f3d-97a9-46f8-a965-e1bcffb45ade;1.0
	confs: [default]
	found com.google.cloud.spark#spark-3.3-bigquery;0.37.0 in central
	found com.google.cloud.spark#spark-bigquery-dsv2-common;0.37.0 in central
	found com.google.cloud.spark#spark-bigquery-connector-common;0.37.0 in central
	found com.google.cloud.spark#bigquery-connector-common;0.37.0 in central
	found com.google.api.grpc#grpc-google-cloud-bigquerystorage-v1;3.3.1 in central
	found io.grpc#grpc-api;1.62.2 in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found com.google.errorprone#error_prone_annotations;2.23.0 in central
	found io.grpc#grpc-stub;1.62.2 in central
	found io.grpc#grpc-protobuf;1.62.2 in central
	found com.google.api.grpc#proto-google-common-protos;2.36.0 in centr

	org.eclipse.collections#eclipse-collections;11.1.0 from central in [default]
	org.eclipse.collections#eclipse-collections-api;11.1.0 from central in [default]
	org.json#json;20231013 from central in [default]
	org.slf4j#slf4j-api;2.0.9 from central in [default]
	org.threeten#threeten-extra;1.7.2 from central in [default]
	org.threeten#threetenbp;1.6.8 from central in [default]
	:: evicted modules:
	com.google.code.gson#gson;2.10.1 by [com.google.code.gson#gson;2.9.1] in [default]
	org.apache.arrow#arrow-format;15.0.1 by [org.apache.arrow#arrow-format;15.0.0] in [default]
	org.apache.arrow#arrow-memory-core;15.0.1 by [org.apache.arrow#arrow-memory-core;15.0.0] in [default]
	org.checkerframework#checker-qual;3.10.0 by [org.checkerframework#checker-qual;3.42.0] in [default]
	com.google.guava#failureaccess;1.0.2 by [com.google.guava#failureaccess;1.0.1] in [default]
	org.checkerframework#checker-qual;3.41.0 by [org.checkerframework#checker-qual;3.42.0] in [default]
	io.perfmark#perfmark-a

In [3]:
df = spark.read \
        .format("bigquery") \
        .option("table", "airbnb-de-project.airbnb_dw.airbnb_data") \
        .load()

24/04/14 18:23:14 WARN DefaultCredentialsProvider: You are authenticating using user credentials. For production, we recommend using service account credentials.

To learn more about service account credentials, see http://cloud.google.com/docs/authentication/external/set-up-adc-on-cloud


In [4]:
df.printSchema()

root
 |-- listing_id: long (nullable = true)
 |-- host_id: long (nullable = true)
 |-- host_since: string (nullable = true)
 |-- host_response_rate: double (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_total_listings_count: long (nullable = true)
 |-- host_has_profile_pic: string (nullable = true)
 |-- host_identity_verified: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- district: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- property_type: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- accommodates: long (nullable = true)
 |-- bedrooms: long (nullable = true)
 |-- price: long (nullable = true)
 |-- review_scores_rating: long (nullable = true)
 |-- review_scores_accuracy: long (nullable = true)
 |-- review_scores_cleanliness: long (nullable = true)
 |-- review_scores_checkin: long (nullable = true)
 |-- 

In [5]:
df.show()

24/04/14 18:23:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+---------+----------------+------------------+-----------------+-------------------------+--------------------+----------------------+-------------+--------+--------------+---------+---------+--------------------+------------+------------+--------+-----+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+---------+---------+-----------+----------------+--------------------+
|listing_id|  host_id|      host_since|host_response_rate|host_is_superhost|host_total_listings_count|host_has_profile_pic|host_identity_verified|neighbourhood|district|          city| latitude|longitude|       property_type|   room_type|accommodates|bedrooms|price|review_scores_rating|review_scores_accuracy|review_scores_cleanliness|review_scores_checkin|review_scores_communication|review_scores_location|review_scores_value|price_usd|review_id|reviewer_id|     review_date|listing_review_count|


In [6]:
# Convert price to USD

# Conversion rates
conversion_rates = {
    'Paris': 1.07,
    'New York': 1,
    'Bangkok': 0.027,
    'Rio de Janeiro': 0.20,
    'Sydney': 0.65,
    'Istanbul': 0.031,
    'Rome': 1.07,
    'Hong Kong': 0.13,
    'Mexico City': 0.060,
    'Cape Town': 0.053
}

# Define a UDF (user-defined function) to calculate the converted price
@udf(FloatType())
def convert_currency(city, price):
    return price * conversion_rates.get(city, 1)

# Apply the UDF to create a new column "price_usd"
df = df.withColumn("price_usd", convert_currency(col("city"), col("price")))

# Display the updated dataframe
df.head(1)

[Row(listing_id=37200137, host_id=77212938, host_since='2018-06-12T00:00', host_response_rate=1.0, host_is_superhost='t', host_total_listings_count=2, host_has_profile_pic='t', host_identity_verified='t', neighbourhood='Joa', district=None, city='Rio de Janeiro', latitude=-23.01489, longitude=-43.29168, property_type='Entire condominium', room_type='Entire place', accommodates=6, bedrooms=3, price=480, review_scores_rating=100, review_scores_accuracy=10, review_scores_cleanliness=10, review_scores_checkin=10, review_scores_communication=10, review_scores_location=10, review_scores_value=10, price_usd=96.0, review_id=607725284, reviewer_id=107166188, review_date='2022-02-18T00:00', listing_review_count=44)]

In [7]:
df.show(5)

+----------+--------+----------------+------------------+-----------------+-------------------------+--------------------+----------------------+-------------+--------+--------------+---------+---------+------------------+------------+------------+--------+-----+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+---------+---------+-----------+----------------+--------------------+
|listing_id| host_id|      host_since|host_response_rate|host_is_superhost|host_total_listings_count|host_has_profile_pic|host_identity_verified|neighbourhood|district|          city| latitude|longitude|     property_type|   room_type|accommodates|bedrooms|price|review_scores_rating|review_scores_accuracy|review_scores_cleanliness|review_scores_checkin|review_scores_communication|review_scores_location|review_scores_value|price_usd|review_id|reviewer_id|     review_date|listing_review_count|
+-----

In [8]:
df.select('city', 'price', 'price_usd').show(10)

+--------------+-----+---------+
|          city|price|price_usd|
+--------------+-----+---------+
|Rio de Janeiro|  480|     96.0|
|Rio de Janeiro| 1000|    200.0|
|Rio de Janeiro| 1254|    250.8|
|Rio de Janeiro| 1254|    250.8|
|Rio de Janeiro| 1254|    250.8|
|Rio de Janeiro| 1254|    250.8|
|Rio de Janeiro| 1254|    250.8|
|Rio de Janeiro| 1254|    250.8|
|Rio de Janeiro| 1500|    300.0|
|Rio de Janeiro| 6500|   1300.0|
+--------------+-----+---------+
only showing top 10 rows



In [9]:
# Calculate the average price_usd
avg_price_usd = df.groupBy().agg(avg('price_usd').alias('average_price_usd'))

# Show the result
avg_price_usd.show()

+-----------------+
|average_price_usd|
+-----------------+
|88.78709692960278|
+-----------------+



In [10]:

key_file_path = '/home/sal/git/data-engineering-capstone/spark/keys/my-creds.json'

# Define the Google Cloud project ID
project_id = "airbnb-dezoomcamp"

# Set the BigQuery dataset and table name
dataset_name = "airbnb_dw"
table_name = "airbnb_data_spark"

# Write the DataFrame to BigQuery
df.write \
    .format("bigquery") \
    .option("temporaryGcsBucket", "airbnb-data-lake") \
    .option("project", project_id) \
    .option("dataset", dataset_name) \
    .option("table", table_name) \
    .option("credentialsFile", key_file_path) \
    .save()



Py4JJavaError: An error occurred while calling o76.save.
: java.io.UncheckedIOException: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at com.google.cloud.spark.bigquery.SparkBigQueryUtil.createGcsPath(SparkBigQueryUtil.java:129)
	at com.google.cloud.spark.bigquery.write.BigQueryWriteHelper.<init>(BigQueryWriteHelper.java:89)
	at com.google.cloud.spark.bigquery.write.BigQueryDeprecatedIndirectInsertableRelation.insert(BigQueryDeprecatedIndirectInsertableRelation.java:41)
	at com.google.cloud.spark.bigquery.write.CreatableRelationProviderHelper.createRelation(CreatableRelationProviderHelper.java:58)
	at com.google.cloud.spark.bigquery.v2.Spark31BigQueryTableProvider.createRelation(Spark31BigQueryTableProvider.java:65)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:48)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apac

In [ ]:
# Write transformed data back to BigQuery
df.write \
    .format("bigquery") \
    .option("temporaryGcsBucket", "airbnb-data-lake") \
    .option("table", "airbnb-de-project.airbnb_dw.airbnb_data_spark") \
    .mode("overwrite") \
    .save()

In [ ]:
spark.stop()